In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# First part of the homework #
find the top news story headline and the summary

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [29]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/Howard/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [5]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [6]:
results = soup.find_all('div', class_="list_text")
date = soup.find('div', class_="list_date").text
news_title = results[0].find('a').text
news_p= soup.find('div', class_="article_teaser_body").text

# Second part of homework (get the featured image url from JPL page)#

In [27]:
url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url2)
# HTML object
html2 = browser.html
base = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
# Parse HTML with Beautiful Soup
soup = bs(html2, 'html.parser')
# find the parent div of the image
img_url = soup.find('div', class_='floating_text_area')
# find the href within the div
ref = img_url.find('a')['href']
# build the final link for the featured image
featured_image_url = base + ref

In [22]:
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

# Third part of homework (use pandas to get table from webpage) #

In [8]:
url3 = 'https://space-facts.com/mars/'

In [10]:
tables = pd.read_html(url3)
table = tables[0]
# found solutions https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas #2192
table.columns = ['Description','Mars']
table

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
table = table.to_html(index=False)

# Mars Hemispheres #

In [30]:
# URL of page to be scraped
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url4)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [31]:
results = soup.find_all('div', class_='item')
title_obj = [i.h3.text for i in results]
href_obj = [base + i.a['href'] + '.tif' for i in results]

In [32]:
hemisphere_image_urls = []
for i in range(len(title_obj)):
    dict = {}
    dict['title'] = title_obj[i]
    dict['img_url'] = href_obj[i]
    hemisphere_image_urls.append(dict)
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space//search/map/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space//search/map/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space//search/map/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space//search/map/Mars/Viking/valles_marineris_enhanced.tif'}]


In [38]:
browser.quit()

In [37]:
hemisphere_image_urls[1]["img_url"]

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space//search/map/Mars/Viking/schiaparelli_enhanced.tif'